<a href="https://colab.research.google.com/github/ozturkcemal/CorkPubCrawl/blob/main/CorkPubCrawl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install ortools openrouteservice

In [2]:
# Step 1: Import libraries
import openrouteservice
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [4]:
# Step 2: Setup your coordinates and API key
coordinates = [
[-8.4773019737901, 51.89801157949557],    #Liberty Bar
[-8.47839267379017, 51.89754580132632],   #Dwyers
[-8.480129616118258, 51.897412860783],    #Costigans
[-8.48210270262587, 51.90122750985092],   #Franciscan Well
[-8.478174116118428, 51.89376597099192],  #Tom Barry's
[-8.470903544953982, 51.901992373046355], #Corner House
[-8.47113337564154, 51.90199549372547],   #Sin E'
[-8.4765895179699, 51.896701021615215],   #An Spailpin Fanach
[-8.47664922081232, 51.89677734873688],   #The Oval
[-8.466700360297953, 51.897178061440265], #Charlies
[-8.470990660298165, 51.89379653393844],  #Fionbarra
[-8.469605244954147, 51.89843912054248]   # The Oliver Plunkett
]

In [5]:
api_key = "5b3ce3597851110001cf6248ac6b2739185944338284b13037a0ba41"
client = openrouteservice.Client(key=api_key)


In [10]:
# Step 3: Get the distance matrix from ORS
matrix = client.distance_matrix(
    locations=coordinates,
    profile='foot-walking',
    #profile='driving-car',
    metrics=['distance'],
    units='m'
)
distance_matrix = matrix['distances']

In [ ]:
# Step 4: Create the data model
def create_data_model():
    data = {}
    data['distance_matrix'] = distance_matrix
    data['num_vehicles'] = 1
    data['depot'] = 0  # Starting point
    return data

data = create_data_model()

print("Distance Matrix:")
for row in distance_matrix:
    print(row)

In [13]:
# Step 5: OR-Tools setup
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                       data['num_vehicles'],
                                       data['depot'])
routing = pywrapcp.RoutingModel(manager)

def distance_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return int(data['distance_matrix'][from_node][to_node])

transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

In [ ]:
# Step 6: Solve and print the solution
solution = routing.SolveWithParameters(search_parameters)

def print_solution(manager, routing, solution):
    print("Optimized route:")
    index = routing.Start(0)
    route = []
    route_distance = 0
    while not routing.IsEnd(index):
        node = manager.IndexToNode(index)
        route.append(node)
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    route.append(manager.IndexToNode(index))
    print(" -> ".join(str(i) for i in route))
    print("Total distance: {:.2f} km".format(route_distance / 1000))

if solution:
    print_solution(manager, routing, solution)
else:
    print("No solution found.")